In [ ]:
!pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu124
!pip install accelerate bitsandbytes
!pip install git+https://github.com/huggingface/transformers
!pip install optimum --no-deps

Looking in indexes: https://download.pytorch.org/whl/nightly/cu124
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-jga0o48u
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-jga0o48u
  Resolved https://github.com/huggingface/transformers to commit df6eee9201e4ba2b80cea021a18e95ada26ca2cc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.44.0.dev0-py3-none-any.whl size=9404002 sha256=17102ef6b4b234e01c3944f359d13f2007cd235cc1e5b44744306cc4d301a061
  Stored in directory: /tmp/pip-ephem-wheel-cache-esyv7mrc/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.43.2
    Uninstalling transformers-4.43.2:
      Successfully unins

In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [ ]:
path: str = "meta-llama/Meta-Llama-3.1-8B"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model_quantized = AutoModelForCausalLM.from_pretrained(
    path,
    quantization_config=quant_config,
    torch_dtype=torch.bfloat16,
    device_map="cuda"
)

model_quantized

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(path)

In [ ]:
prompt = "Are you alive?"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

answer = model_quantized.generate(inputs.input_ids, max_length=10).to(device)
tokenizer.batch_decode(
    answer, skip_special_tokens=True, clean_up_tokenization_spaces=False
)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'Are you alive? Are you aware of your'

In [ ]:
quantized_path: str = "akshathmangudi/llama3.1-8b-quantized"

model_quantized.push_to_hub(quantized_path)
tokenizer.push_to_hub(quantized_path)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/akshathmangudi/llama3.1-8b-quantized/commit/9e5f622bbaa49f0ccb45ab947cbb10ae4f90a2d0', commit_message='Upload tokenizer', commit_description='', oid='9e5f622bbaa49f0ccb45ab947cbb10ae4f90a2d0', pr_url=None, pr_revision=None, pr_num=None)